In [12]:
import numpy as np
from scipy.stats import chi2
from random import random, gauss, gammavariate
from tabulate import tabulate
from math import exp, comb, log, sqrt, erf, pi

## Funciones generales
def generar_muestra(generador, n):
    """
    Genera una muestra de tamaño n de una variable aleatoria
    Args:
        generador:    Generador de la variable aleatoria
        n        :    Tamaño de la muestra
    """
    datos = []
    for _ in range(n):
        datos.append(generador())
    return datos


def generar_binomial(N, P):
    """
    Genera un valor de la variable aleatoria binomial usando el metodo de la 
    transformada inversa
    Args:
        N: Cantidad de ensayos independientes
        P: Probabilidad de exito de ensayo
    """
    prob = (1 - P) ** N
    c = P / (1 - P)
    F = prob
    i = 0
    U = random()
    while U >= F:
        prob *= c * (N-i) / (i+1)
        F += prob
        i += 1
    return i


def generar_exp(Lambda):
    """
    Genera una variable aleatoria exponencial usando el metodo de la transformada
    inversa
    Args:
        Lambda: Parametro de la variable aleatoria exponencial
    """
    U = random()
    return -log(U) / Lambda


def uniforme_d(n, m):
    """
    Genera numeros aleatorios entre n y m (inclusive) 
    """
    U = int(random() * (m - n + 1))  # 0 <= U < (m-n+1) Equivale a 0 <= U <= (m-n)
    U = U + n                        # Equivale a n <= U <= m
    return U


def estimar_proporcion_d(d, generador, n=100):
    """
    Estima la proporción de exito de una bernoulli, con un desvio 
    de estimador acotado
    Args:
        n:       : Tamaño minimo de la muestra
        d        : Cota para la desviacion aceptable del estimador
        generador: Generador de una bernoulli 
    Return:
        p        : Estimacion de la proporcion
        var_m    : Varianza muestral obtenida
        i        : Tamaño de la muestra final
    """
    p = 0
    i = 0
    while i <= n or sqrt(p * (1 - p) / i) > d:
        i += 1
        X = generador()
        p = p + (X - p) / i
    var_m = p * (1 - p)
    return p, var_m, i

## Funciones generales (Test para datos discretos)
def calcular_T(N, p):
    """
    Calcula el estadistico T dado en el test chi-cuadrado de 
    Pearson.
    ¡Ambas entradas deben estar ordenadas de menor a mayor respecto
    al valor de variable aleatoria al que hacen referencia!
    Args:
        N:    Frecuencia observada de la muestra
        p:    Probabilidad de los valores posibles de la variable aleatoria
    """
    n = sum(N)    # Tamaño de la muestra
    T = sum((N - n*p)**2 / (n*p))
    return T


def p_valor_pearson(t, grados_libertad):
    """
    Calcula el p-valor usando los resultados teoricos que dictan:
    Ph0(T >= t) = P(X_k-1 >= t)
    Args:
        t              :    Resultado del estadistico en la muestra
        grados_libertad:    Grados de libertad de la variable chi-cuadrada
    """
    acumulada_chi = chi2.cdf(t, grados_libertad)
    p_valor = 1 - acumulada_chi
    return p_valor


def generar_frecuencia_observada(n, p_masa, k):
    """
    Genera la frecuencia observada usando una variable aleatoria binomial
    Args:
        n     :   Tamaño de la muestra
        p_masa:   Probabilidad masa de los valores posibles de la distribucion 
                  de la hipotesis nula
        k     :   Cantidad total de valores posibles de la variable con distribucion
                  dada en la hipotesis nula
    """
    N = np.zeros(k)
    N[0] = generar_binomial(n, p_masa[0])
    for i in range(1, k-1):
        N[i] = generar_binomial(
            n - sum(N[:i]), p_masa[i] / (1 - sum(p_masa[:i])))
    N[k-1] = n - sum(N[:k-1])
    return np.array(N)


def estimar_p_valor_pearson(t, Nsim, p, k, n):
    """
    Estima el p_valor por medio de una simulacion P(T >= t)
    Args:
        t   :   Valor del estadistico en la muestra original
        Nsim:   Cantidad de simulaciones a realizar
        p   :   Probabilidad masa de los valores posibles de la distribucion 
                de la hipotesis nula
        k   :   Cantidad total de valores posibles de la variable con distribucion
                dada en la hipotesis nula
        n   :   Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        # Frecuencia observada en la simulacion
        N_sim = generar_frecuencia_observada(n, p, k)
        # Estadistico de la simulacion
        t_sim = calcular_T(N_sim, p)
        if t_sim >= t:
            p_valor += 1
    return p_valor / Nsim

## Funciones generales (Test para datos continuos)
def calcular_D(muestra, acumulada_h):
    """
    Calcula el estadistico dado en el test de Kolmogorov-Smirnov
    Args:
        muestra    :    Muestra original de datos
        acumulada_h:    Acumulada de la distribucion de la hipotesis nula
    """
    n = len(muestra)
    m_copy = list.copy(muestra)
    m_copy.sort()
    m_copy = np.array(m_copy)

    # Vectores_auxiliares
    F = np.array(list(map(acumulada_h, m_copy)))
    j = np.array(list(range(1, n+1))) / n
    j_1 = j - 1/n

    m1 = max(j - F)
    m2 = max(F - j_1)
    D = max(m1, m2)
    return D


def acumulada_uniforme(x):
    """
    Acumulada de la distribucion uniforme (U(0, 1))
    """
    if x < 0:
        return 0
    elif x < 1:
        return x
    else:
        return 1


def estimar_p_valor_k(d, Nsim, n):
    """
    Estima el p-valor por medio de simulacion P(D >= d)
    args:
        d   :    Valor del estadistico en la muestra original
        Nsim:    Cantidad de simulaciones a realizar
        n   :    Tamaño de la muestra original
    """
    p_valor = 0
    for _ in range(Nsim):
        muestra_sim = generar_muestra(random, n)    # Muestra de uniformes
        muestra_sim.sort()
        d_sim = calcular_D(muestra_sim, acumulada_uniforme)
        if d_sim >= d:
            p_valor += 1
    return p_valor / Nsim

In [11]:
# Ejercicio 1
N = np.array([120, 114, 92, 85, 34, 33, 45, 11, 5])
n = sum(N)
p = np.array([0.22, 0.2, 0.19, 0.12, 0.09, 0.08, 0.07, 0.02, 0.01])
k = 9

t = calcular_T(N, p)

p_valor = p_valor_pearson(t, grados_libertad = k - 1)

# Simulacion
Nsim = 10**4
p_valor_sim = estimar_p_valor_pearson(t, Nsim, p, k, n)

datos   = [[t, p_valor, p_valor_sim]]
headers = ["Estadistico T", "P-valor por Pearson", "P-valor simulado"]

print(tabulate(datos, headers=headers, tablefmt="pretty"))

+--------------------+---------------------+------------------+
|   Estadistico T    | P-valor por Pearson | P-valor simulado |
+--------------------+---------------------+------------------+
| 15.964309985059911 | 0.04289380378443686 |      0.0443      |
+--------------------+---------------------+------------------+


In [27]:
# Ejercicio 4
def cae_en_elipse():
    """
    Genera la variable aleatoria bernoulli que modela el hecho de que un punto
    aleatorio del cuadrado [-2,2]x[-1,1] caiga en la elipse
    """
    X = np.random.uniform(-2, 2)
    Y = np.random.uniform(-1, 1)
    return int((X/2)**2 + Y**2 <= 1)

L = 0.1
c = 1.96
d = L / (2 * c)

p_est = estimar_proporcion_d(d, cae_en_elipse) 

estimacion_pi  = p_est[0]
var_est = p_est[1]
n = p_est[2]
intervalo = [estimacion_pi - c * sqrt(var_est/n), estimacion_pi + c * sqrt(var_est/n)]

data    = [[pi*2, estimacion_pi*8, n]]
headers = ["Valor del Area", "Estimacion del Area", "Numero de itraciones"]

print(tabulate(data, headers=headers, tablefmt="pretty"))
print(f"Intervalo de confianza: {intervalo}")

+-------------------+---------------------+----------------------+
|  Valor del Area   | Estimacion del Area | Numero de itraciones |
+-------------------+---------------------+----------------------+
| 6.283185307179586 |  6.153846153846156  |         273          |
+-------------------+---------------------+----------------------+
Intervalo de confianza: [0.719251255903911, 0.819210282557628]
